In [1]:
# Install requirements
# drop the -q if you want to se the results
! pip install -qr requirements.txt

In [2]:
# Check the API KEYs
import os

config = {}

if os.path.exists('.env'):
    from dotenv import dotenv_values
    config = dict(dotenv_values(".env"))
else:
    # Ask about the Qdrant API (get one in cloud.qdrant.io)
    config['QDRANT_API_KEY'] = input("What is your Qdrant API key?")
    config['QDRANT_CLUSTER'] = input("What is your Qdrant cluster?")
    config['QDRANT_COLLECTION'] = input("What is your Qdrant collection?")   

In [3]:
# Prompt
def create_prompt(context, previous_version, current_version):
    prompt = """
You are an experienced C++ developer and a frequent contributor to Stack Overflow. 
Your task is to provide a comprehensive, well-structured, and insightful answer to the question below based on the provided context.

### Context Information:
{context}

### Question:
What are the key benefits, challenges, and best practices associated with migrating from C++{previous_version} to C++{current_version}, as well as performing source code rejuvenation?

### Instructions:
1. Use only the context provided to support your answer, ensuring relevance and clarity.
2. Be specific and provide actionable, real-world examples when possible.
3. Answer each aspect of the question: benefits, challenges, and best practices.
4. Structure your answer clearly, using bullet points or numbered lists.
5. If the context does not provide enough information, state 'I don't know'.

### Response Format:
--------------------------------------------------------------
**Benefits:**
1. [Provide a benefit of migration here.]
2. [Provide another benefit of migration here.]
3. [Provide a third benefit of migration here.]

**Challenges:**
1. [Provide a challenge of migration here.]
2. [Provide another challenge of migration here.]
3. [Provide a third challenge of migration here.]

**Best Practices for Source Code Rejuvenation:**
1. [Provide a practice for source code rejuvenation here.]
2. [Provide another practice for source code rejuvenation here.]
3. [Provide a third practice for source code rejuvenation here.]

--------------------------------------------------------------
Ensure that the response is concise, actionable, and well-formatted. """

    return prompt.format(context=context,
                         current_version=current_version,
                         previous_version=previous_version
                        )

In [4]:
# Config search on Qdrant
from qdrant_client import QdrantClient

client = QdrantClient(
    url=config['QDRANT_CLUSTER'],
    https=True,
    api_key=config['QDRANT_API_KEY']
)

/home/charles/unb-alana/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Define a basic search 
def search(query):
    results = client.query(
        collection_name=config['QDRANT_COLLECTION'],
        query_text=query,
        limit=5
    )
    return results

In [6]:
# A silly test
results = search("What are the main benefits and challenges of migrating from C++11 to C++14?")
for i, result in enumerate(results):
    print(f"{i+1}) {result.document}")
    print(f"   postID: {result.metadata['postId']}")
    print(f"   viewCount: {result.metadata['viewCount']}")    
    print(f"   qdrant score: {result.score}")    

1) Are there any plans for C++14 or later for standardized backtrace and other debugging utilities? Are there any plans for standardized backtrace and other debugging utilities to be added to C++14 (or above), or will it continue to be a functionality added by OS or compiler? 
   postID: 21819017
   viewCount: 480
   qdrant score: 0.8959615
2) How many versions of the C++ Standards are there? In C there's C89, C99, and C11; what about C++? Is there any upgrade since it came out?  
   postID: 3940389
   viewCount: 14367
   qdrant score: 0.88990366
3) How are you using C++11 today? This is a question in two parts, the first is the most important and concerns now:   Are you following the design and evolution of C++11?  What blogs, newsgroups, committee papers, and other resources do you follow? Even where you're not using any new features, how have they affected your current choices? What new features are you using now, either in production or otherwise?   The second part is a follow-up, 

In [7]:
# Make a full prompt
context = '\n\n'.join([x.metadata['document'] for x in results])
prompt = create_prompt(context=context, previous_version="11", current_version="13")
print(prompt)


You are an experienced C++ developer and a frequent contributor to Stack Overflow. 
Your task is to provide a comprehensive, well-structured, and insightful answer to the question below based on the provided context.

### Context Information:
Are there any plans for C++14 or later for standardized backtrace and other debugging utilities? Are there any plans for standardized backtrace and other debugging utilities to be added to C++14 (or above), or will it continue to be a functionality added by OS or compiler? 

How many versions of the C++ Standards are there? In C there's C89, C99, and C11; what about C++? Is there any upgrade since it came out?  

How are you using C++11 today? This is a question in two parts, the first is the most important and concerns now:   Are you following the design and evolution of C++11?  What blogs, newsgroups, committee papers, and other resources do you follow? Even where you're not using any new features, how have they affected your current choices? W

In [8]:
# Call local ollama
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])
print(response.message.content)

**Benefits:**

1. **Improved Code Quality**: Migrating from C++11 to C++14 (or later) allows you to leverage new language features, such as `auto` keyword, range-based for loops, and user-defined literals, which can improve code readability, maintainability, and performance.
2. **Enhanced Security**: Adopting the latest C++ standards enables you to take advantage of new security-related features, like stacktrace handling and memory safety guarantees, reducing the risk of memory-related bugs and vulnerabilities.
3. **Better Library Support**: As libraries and frameworks continue to evolve, migrating to a newer C++ standard ensures compatibility with updated library versions, which can lead to improved performance, functionality, and maintainability.

**Challenges:**

1. **Compiler Support**: The availability of compiler support for the latest C++ standards (e.g., GCC 7+, Clang 4+) might be limited on certain platforms or architectures, potentially hindering migration efforts.
2. **Legac